# The steps below will create and populate an FTS database, which we may want to do every once in a while just to be sure we're current.

Below will create a new fts5_new.db that is empty -- it is a standard fts database and contains content.

In [1]:
import sqlite3
#db = sqlite3.connect('mylistmanager_s.db')
fts = sqlite3.connect('fts5.db')
fts_c = fts.cursor()
# uncomment below when you want to recreate the fts5 database
#fts.execute('CREATE VIRTUAL TABLE fts USING fts5 (title, note, lm_id UNINDEXED);')
# not sure the below is necessary - deleted items just don't find matches in mylismanager_s.db
#fts.execute('CREATE VIRTUAL TABLE fts USING fts5 (title, note, lm_id UNINDEXED, deleted);') ##?

This is how you create a new fts database that will be named whatever you named it above

In [4]:
#c = db.cursor()
#fts_c = fts.cursor()

from lmdb_s import *
from time import sleep
tasks = local_session.query(Task)
for task in tasks:
    if task.deleted:
        continue
    title = task.title
    note = task.note
    if not note:
        note = ""
    title = title.replace("'", "''")
    note = note.replace("'", "''")
    try:
        # the title field is field 0 and the note field is field 1
        fts_c.execute(f"INSERT INTO fts (title, note, lm_id) VALUES (\'{title}\',\'{note}\', {task.id});")
    except Exception as e:
        print(f"INSERT INTO fts (title, note, lm_id) VALUES (\'{title}\',\'{note}\', {task.id});")
        print(e)
        
    sleep(.01) 
    

fts.commit()
print("done")

done


This demonstrates searching and retrieving ids using the FTS5 virtual table to retrieve matches

In [2]:
fts_c.execute("SELECT lm_id FROM fts WHERE fts MATCH 'norm';")
rows = fts_c.fetchall()
print(rows)
#fts_c.close()
#fts.close()

[(3,), (22,), (229,), (1046,), (1557,), (2002,), (2664,), (3111,), (3169,), (3249,), (3427,), (3487,), (4096,), (4752,), (4795,), (5065,)]


This demonstrates highlighting the title (field is 0)

In [4]:
fts = sqlite3.connect('fts5.db')
fts_c = fts.cursor()
fts_c.execute("SELECT lm_id, highlight(fts, 0, '[',']') FROM fts WHERE fts MATCH 'norm' AND lm_id=229;")
rows = fts_c.fetchall()
print(rows)

[(229, '[Norm] St. Landau Feb. 12~')]


This demonstrates highlighting the note (field = 1)

In [11]:
fts_c.execute("SELECT lm_id, highlight(fts, 1, '[',']') FROM fts WHERE fts MATCH 'norm' AND lm_id=5065;")
rows = fts_c.fetchall()
print(rows)

[(5065, '[Norm] is a really good guy.')]
